In [ ]:
Практическая работа № 6

Выполнил студент группы ББМО-01-23: Буланов Андрей Алексеевич

Цель работы:

1. Загрузить несколько моделей, обученных на датасете MNIST.
2. Изучить теоретические основы атаки по переносу.
3. Реализовать атаку FGSM на одну модель и проверить, как противоречивые примеры влияют на
другую модель.
4. Оценить точность обеих моделей на противоречивых примерах и проанализировать
переносимость атак.

In [1]:
# Импорт всех необходимых библиотек
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Загрузка и подготовка данных
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0
train_labels, test_labels = to_categorical(train_labels), to_categorical(test_labels)

# Модель 1: Полносвязная нейронная сеть
model1 = Sequential([Flatten(input_shape=(28, 28)), Dense(128, activation='relu'), Dense(10, activation='softmax')])
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1.fit(train_images, train_labels, epochs=5)
model1.save('mnist_model_1.h5')

# Модель 2: Свёрточная нейронная сеть (CNN)
model2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')])
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=5)
model2.save('mnist_model_2.h5')

Epoch 1/5


c:\Users\Xazerer(CS24)\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8796 - loss: 0.4333
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9637 - loss: 0.1218
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9752 - loss: 0.0809
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9821 - loss: 0.0587
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9861 - loss: 0.0440


c:\Users\Xazerer(CS24)\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9115 - loss: 0.3058
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9842 - loss: 0.0544
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9905 - loss: 0.0320
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9933 - loss: 0.0214
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9959 - loss: 0.0130


In [3]:
# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    # Применение знака градиента к изображению
    perturbed_image = image + epsilon * np.sign(gradient)
    perturbed_image = np.clip(perturbed_image, 0, 1)  # Ограничение значений в диапазоне [0, 1]
    return perturbed_image

# Генерация противоречивых примеров
def generate_fgsm_adversarial(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape(1, 28, 28, 1), dtype=tf.float32) # Конвертирует размер под формат модели
        label = tf.convert_to_tensor(labels[i].reshape(1, -1), dtype=tf.float32) # Конвертируем one-hot вектор в индекс

        # Вычисление градиента
        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)

        gradient = tape.gradient(loss, image)
        adv_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
        adversarial_images.append(adv_image.reshape(28, 28))

    return np.array(adversarial_images)

# Создание противоречивых примеров для первой модели
epsilon = 0.1
adversarial_images_model1 = generate_fgsm_adversarial(model1, test_images, test_labels, epsilon)

In [4]:
# Оценка первой модели на противоречивых примерах
test_labels_argmax = np.argmax(test_labels, axis=1) # Преобразование onehot меток в целые числа
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels)
print(f'Accuracy of model1 on adversarial examples: {acc1}')

# Оценка второй модели на противоречивых примерах (перенос атаки)
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1, 28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped, test_labels)
print(f'Accuracy of model2 on adversarial examples from model1: {acc2}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step - accuracy: 0.0772 - loss: 6.5305
Accuracy of model1 on adversarial examples: 0.10010000318288803
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9543 - loss: 0.1526
Accuracy of model2 on adversarial examples from model1: 0.9616000056266785


In [5]:
# Генерация противоречивых примеров для второй модели
adversarial_images_model2 = generate_fgsm_adversarial(model2, test_images.reshape(-1, 28, 28, 1), test_labels, epsilon)
# Оценка первой модели на противоречивых примерах второй модели
loss3, acc3 = model1.evaluate(adversarial_images_model2.reshape(-1, 28, 28), test_labels)
print(f'Accuracy of model1 on adversarial examples from model2: {acc3}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step - accuracy: 0.9575 - loss: 0.1356
Accuracy of model1 on adversarial examples from model2: 0.9632999897003174


Вывод:

Точность модели на противоречивых примерах имеет низкую точность в размере ~10%, как обычная имеет 96.1%. Атаки моделей показали небольшое снижение точности, что говорит об устойчивости модели к атаке, которая создана для другой модели.